In [ ]:
!pip install -r requirements.txt

## Importing Packages

In [73]:
import os
import time
import logging
import pandas as pd
import sqlite3
from Bio import Entrez
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, set_seed
from sentence_transformers import SentenceTransformer, util
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)


import nltk

import torch
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

## Importing Data

Procedures used during data ingestion

In [77]:
def preprocess_data(file_paths):
    dataframes = []
    for file in file_paths:
        df = pd.read_csv(file, delimiter='\t', encoding='ISO-8859-1', on_bad_lines='skip')
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
        non_numeric_cols = df.select_dtypes(exclude=['float64', 'int64']).columns
        df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
        df[non_numeric_cols] = df[non_numeric_cols].fillna('')
        dataframes.append(df)
    return dataframes

def combine_text_data(dataframes, excel_df, pubmed_df):
    """Combine text data from multiple sources into a single list."""
    all_text_data = [text for df in dataframes for col in df.select_dtypes(include=[object]) for text in df[col].dropna().tolist()]
    
    # Extract text data from the Excel DataFrame
    for col in excel_df.select_dtypes(include=[object]).columns:
        all_text_data += excel_df[col].dropna().tolist()
    
    # Extract text data from the PubMed abstracts
    all_text_data += pubmed_df['Abstract'].tolist()
    
    return all_text_data


Data Ingestion

In [80]:
import os
import pandas as pd
import sqlite3
import logging

# Configure logging with timestamps
logging.basicConfig(filename='data_processing.log', level=logging.ERROR, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Folder containing the text files (same folder as the Jupyter notebook)
data_folder = os.getcwd()

# List all .txt files in the current directory
file_paths = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith('.txt')]

# Function to preprocess data with encoding handling
def preprocess_data(file_paths):
    dataframes = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path, delimiter='\t', encoding='utf-8-sig', on_bad_lines='skip')  # Skip problematic lines
            dataframes.append(df)
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, delimiter='\t', encoding='ISO-8859-1', on_bad_lines='skip')
                dataframes.append(df)
            except Exception as e:
                logging.error(f"Error reading {file_path}: {e}")
        except Exception as e:
            logging.error(f"Error reading {file_path}: {e}")
    return dataframes

# Load and preprocess data
dataframes = preprocess_data(file_paths)

for i, df in enumerate(dataframes):
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    non_numeric_cols = df.select_dtypes(exclude=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
    df[non_numeric_cols] = df[non_numeric_cols].fillna('')
    dataframes[i] = df

# SQL query with corrected join conditions
query = '''
SELECT p.*, a.*, ad.*, t.*, msl.*
FROM table_6 p
LEFT JOIN table_2 a ON p.ApplNo = a.ApplNo
LEFT JOIN table_1 ad ON a.ApplNo = ad.ApplNo
LEFT JOIN table_10 t ON p.ApplNo = t.ApplNo
LEFT JOIN table_4 msl ON p.ApplNo = msl.ApplNo
LEFT JOIN table_5 ms ON msl.MarketingStatusID = ms.MarketingStatusID
'''

# Database connection and query execution
try:
    with sqlite3.connect('database.db') as conn:
        for i, df in enumerate(dataframes):
            table_name = f"table_{i}"
            df.to_sql(table_name, conn, index=False, if_exists='replace')

        final_df = pd.read_sql_query(query, conn)

        # Load the Excel file from the same folder as the Jupyter Notebook
        excel_file_path = os.path.join(data_folder, 'EMA - medicines_output_medicines_en.xlsx')
        excel_df = pd.read_excel(excel_file_path)

except (sqlite3.Error, Exception) as e:
    logging.error(f"Database error: {e}")
    final_df = pd.DataFrame()  # Empty DataFrame in case of error
    excel_df = pd.DataFrame()  # Empty DataFrame in case of error



In [82]:
from Bio import Entrez
import pandas as pd
import time
from tqdm import tqdm
from urllib.error import HTTPError

Entrez.email = "giuseppe.farrugia.14@um.edu.mt"
search_term = "approved drug"  # Search term to fetch relevant articles
retmax = 10000  # Increase retrieval size to 5000

# Define date range for the last 5 years
current_year = 2024
start_year = current_year - 5

# Fetch all article IDs
handle = Entrez.esearch(db="pubmed", term=search_term, retmax=retmax, mindate=f"{start_year}-01-01", maxdate=f"{current_year}-12-31")
record = Entrez.read(handle)
id_list = record["IdList"]

def fetch_article_metadata(article_id):
    try:
        handle = Entrez.efetch(db="pubmed", id=article_id, rettype="abstract", retmode="xml")
        article_record = Entrez.read(handle)
        return article_record
    except HTTPError as e:
        print(f"HTTPError for article ID {article_id}: {e}")
        return None

# Batch fetching to avoid overwhelming Entrez
batch_size = 10
articles_metadata = []
for i in tqdm(range(0, len(id_list), batch_size), desc="Fetching article metadata"):
    batch_ids = id_list[i:i+batch_size]
    for article_id in batch_ids:
        metadata = fetch_article_metadata(article_id)
        if metadata:
            articles_metadata.append(metadata)
        time.sleep(0.1)  # To avoid hitting Entrez's rate limit

def extract_info(article_metadata):
    article_info = {}
    # Check if 'PubmedArticle' list is not empty
    if 'PubmedArticle' in article_metadata and article_metadata['PubmedArticle']:
        article_info['Title'] = article_metadata['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
        if 'Abstract' in article_metadata['PubmedArticle'][0]['MedlineCitation']['Article']:
            article_info['Abstract'] = article_metadata['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
        else:
            article_info['Abstract'] = "No abstract available"
    else:
        article_info['Title'] = "No title available"
        article_info['Abstract'] = "No abstract available"
    return article_info

# Adding progress bar for extraction as well
articles_info = []
for metadata in tqdm(articles_metadata, desc="Extracting article info"):
    articles_info.append(extract_info(metadata))

pubmed_df = pd.DataFrame(articles_info)
#print(pubmed_df.head())


Fetching article metadata:  70%|███████   | 705/1000 [1:30:52<39:22,  8.01s/it]  

HTTPError for article ID 39250989: HTTP Error 400: Bad Request


Extracting article info: 100%|██████████| 9998/9998 [00:00<00:00, 318376.57it/s]


In [84]:
# Combine all textual data into a single list
all_text_data = combine_text_data(dataframes, excel_df, pubmed_df)
#print(f"Extracted {len(all_text_data)} entries from all data sources.")

## Generating Embeddings

Procedures for embeddings

In [88]:
# Procedure for Optimising the Embedding batch size
def measure_processing_time(batch_size):
    start_time = time.time()
    pubmed_embeddings = []
    for i in tqdm(range(0, len(all_text_data), batch_size)):
        batch = all_text_data[i:i + batch_size]
        embeddings = model.encode(batch, convert_to_numpy=True)
        pubmed_embeddings.append(embeddings)
    pubmed_embeddings = np.vstack(pubmed_embeddings)
    end_time = time.time()
    return end_time - start_time


Code of embedding in Sentence Transformers

In [91]:
# Generate embeddings in batches using the standard tqdm progress bar
model = SentenceTransformer('all-MiniLM-L6-v2')
batch_size = 500
pubmed_embeddings = []

for i in tqdm(range(0, len(all_text_data), batch_size)):
    batch = all_text_data[i:i + batch_size]
    embeddings = model.encode(batch, convert_to_numpy=True)
    pubmed_embeddings.append(embeddings)

pubmed_embeddings = np.vstack(pubmed_embeddings)
embedding_dim = pubmed_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
  0%|          | 0/4292 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 1/4292 [00:00<49:58,  1.43it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 2/4292 [00:01<39:20,  1.82it/s]

...

100%|█████████▉| 4290/4292 [1:02:37<00:29, 14.82s/it]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

100%|█████████▉| 4291/4292 [1:02:53<00:15, 15.00s/it]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 4292/4292 [1:03:03<00:00,  1.13it/s]


## Dense & Sparse Retrievals

Procedures for Retrieval Methods

In [95]:
# Dense retrieval with FAISS
def add_to_faiss_index(index, data, batch_size):
    for i in tqdm(range(0, len(data), batch_size), desc="Adding to FAISS index"):
        index.add(data[i:i + batch_size])

def retrieve_dense(query, index, model, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    return distances[0], indices[0]

def retrieve_sparse(query, tfidf_matrix, vectorizer, top_k=5):
    query_vec = vectorizer.transform([query])
    scores = np.dot(tfidf_matrix, query_vec.T).toarray()
    top_k_indices = np.argsort(scores, axis=0)[-top_k:][::-1].flatten()
    return top_k_indices


FAISS Indexing

In [98]:
add_to_faiss_index(index, pubmed_embeddings, batch_size)
print(f"FAISS index size: {index.ntotal}")

# Sparse retrieval using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(all_text_data)


Adding to FAISS index: 100%|██████████| 4292/4292 [00:02<00:00, 1623.41it/s]


FAISS index size: 2145850


Combining Retrievals

In [103]:
def combine_retrievals(dense_indices, sparse_indices, all_text_data, max_contexts=5):
    """Combine dense and sparse retrieval results, filter meaningful contexts, and return the filtered contexts."""
    # Extract the second element from the dense_indices tuple
    dense_indices = dense_indices[1]
    
    # Print statements for debugging
    #print("Dense indices after extraction:", dense_indices)
    #print("Sparse indices before conversion:", sparse_indices)

    try:
        dense_indices = set(map(int, dense_indices))
        sparse_indices = set(map(int, sparse_indices))
    except TypeError as e:
        print("TypeError during conversion:", e)
        return []

    unique_indices = list(dense_indices | sparse_indices)
    
    # Ensure unique_indices is within valid range
    valid_contexts = []
    seen_contexts = set()
    for idx in unique_indices:
        if idx < len(all_text_data):
            context = all_text_data[idx]
            if context not in seen_contexts:
                seen_contexts.add(context)
                valid_contexts.append(context)
    
    # Print the valid contexts for debugging
    #print("Valid contexts:", valid_contexts)

    return valid_contexts[:max_contexts]


## GPT2 Repsonse Generation

In [140]:
import re

def generate_response(query, dense_indices, sparse_indices, all_text_data, max_context_length=512):
    """Generate a response based on the given query and combined retrieval results."""
    valid_contexts = combine_retrievals(dense_indices, sparse_indices, all_text_data, max_contexts=5)

    if not valid_contexts:
        valid_contexts = ["Model couldn't generate results."]

    combined_context = "\n\n".join(valid_contexts)
    query_with_context = f"Based on the information below, provide a concise and informative answer in about 200 words.\n\n{combined_context}\n\nQuestion: {query}\nAnswer:"

    inputs = tokenizer.encode_plus(
        query_with_context,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=max_context_length
    )
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    output_ids = gen_model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=input_ids.shape[1] + 100,
    pad_token_id=tokenizer.eos_token_id,  # Set explicit pad token ID
    num_beams=5,  # Beam search with 5 beams
    no_repeat_ngram_size=2
    )

    
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    # Remove unnecessary characters
    generated_text = re.sub(r'[^a-zA-Z0-9\s,.!?\'"-]', '', generated_text)

    # Ensure proper sentence formation and truncate to around 200 words
    sentences = generated_text.split('. ')
    truncated_response = '. '.join(sentences[:6])  # Adjust number of sentences as needed
    if len(truncated_response.split()) > 200:
        truncated_response = ' '.join(truncated_response.split()[:200]) + '...'
    if truncated_response and truncated_response[-1] != '.':
        truncated_response += '.'

    # Format the response
    formatted_response = f"**Question:** {query}\n\n"
    formatted_response += f"**Answer:**\n\n"
    formatted_response += f"{truncated_response}"

    return formatted_response

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gen_model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set the padding token

# User input for the query
query = input("Enter your query: ")

# Retrieve relevant documents
dense_results = retrieve_dense(query, index, model, top_k=10)
sparse_results = retrieve_sparse(query, tfidf_matrix, vectorizer, top_k=10)

# Combine retrieval results and generate response
response = generate_response(query, dense_results, sparse_results, all_text_data)
print(response)


Enter your query:  What are the common side effects of aspirin


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**Question:** What are the common side effects of aspirin

**Answer:**

Based on the information below, provide a concise and informative answer in about 200 words.

OXYCODONE AND ASPIRIN

ASPIRIN

Question What are the common side effects of aspirin
Answer Aspirin is an anticonvulsant. It is used to treat a wide range of medical conditions, including asthma, rheumatoid arthritis, psoriasis, Crohn's disease, ulcerative colitis, and many other conditions. However, it is also used as an antihistamine, anti-oxidant, antispasmodic and antifungal agent. In addition, aspirin has been shown to reduce the risk of heart attack and stroke. As.
